In [2]:
# USE THIS TO BYPASS KERNEL CRASHING
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

## IMPORT LIBRARIES


In [1]:
# pip install shimmy
# pip install stable-baselines3[extra]
# pip install gym[atari]
# pip install gym[accept-rom-license]
# pip install ale-py gym[atari,accept-rom-license]

In [3]:
import os 
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.evaluation  import evaluate_policy
from stable_baselines3.common.vec_env import VecFrameStack


## TEST ENVIORNMENT 

http://www.atarimania.com/roms/Roms.rar

In [4]:
from gym.envs import registry

# Get all environment IDs
env_ids = list(registry.keys())

# Display only the first 10 IDs
print("\n".join(env_ids[:10]))
print(f"... and {len(env_ids) - 10} more environments.")


ALE/Adventure-v5
ALE/Adventure-ram-v5
ALE/AirRaid-v5
ALE/AirRaid-ram-v5
ALE/Alien-v5
ALE/Alien-ram-v5
ALE/Amidar-v5
ALE/Amidar-ram-v5
ALE/Assault-v5
ALE/Assault-ram-v5
... and 986 more environments.


In [5]:
environment_name = 'Breakout-v0'
env = gym.make('ALE/Breakout-v5', render_mode='human')  

In [6]:
env.reset()

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 {'lives': 5, 'episode_frame_number': 0, 'frame_number': 0})

In [7]:
env.action_space, env.observation_space

(Discrete(4), Box(0, 255, (210, 160, 3), uint8))

In [ ]:
episodes = 5

for episode in  range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while  not done:
        env.render()
        action = env.action_space.sample() # take random action
        obs_, reward, done, truncated, info = env.step(action)
        score += reward
     
    print('Episode:{} Score:{}'.format(episode, score))
    
# env.close()

## VECTORISING AND TRAINING THE MODEL

In [8]:
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
# env = make_atari_env('ALE/Breakout-v5', n_envs=4, seed=0)
# env = VecFrameStack(env, n_stack=4) 
env = gym.make('ALE/Breakout-v5', render_mode='human')  
env = DummyVecEnv([lambda: env])

c:\Users\ASUS\anaconda3\envs\rl_breakout_env\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [10]:
log_path = os.path.join('Training','logs')
log_path

'Training\\logs'

In [17]:
model = A2C(policy='MlpPolicy',
            verbose=1,
            env=env,
            tensorboard_log=log_path,
            device=device)

model.learn(total_timesteps =100000)

Using cuda device
Wrapping the env in a VecTransposeImage.
Logging to Training\logs\A2C_5
-------------------------------------
| time/                 |           |
|    fps                | 11        |
|    iterations         | 100       |
|    time_elapsed       | 43        |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.23     |
|    explained_variance | -2.25e-05 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -0.00393  |
|    value_loss         | 1.27e-05  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 12        |
|    iterations         | 200       |
|    time_elapsed       | 77        |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -1.21     |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    

## SAVE THE MODEL

In [12]:
A2C_path = os.path.join('Training', 'Models', 'A2C_MODEL_BREAKOUT')

In [ ]:
model.save(A2C_path)

In [ ]:
del model

In [13]:
model =  A2C.load(A2C_path, env)

Wrapping the env in a VecTransposeImage.


c:\Users\ASUS\anaconda3\envs\rl_breakout_env\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


## TEST THE MODEL

In [17]:
evaluate_policy(model, env, 2, render= False)

c:\Users\ASUS\anaconda3\envs\rl_breakout_env\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(np.float64(0.0), np.float64(0.0))

In [20]:
episodes = 5

for episode in range(1, episodes+1):
    
    done = False
    obs = env.reset()
    score = 0
    
    while not done :
        
        env.render()
        action,_ = model.predict(obs)
        obs, reward, done,  info = env.step(action)
        score += reward
    print(f"episode : {episode}, score : {score}")  
    
# env.close()      
        

episode : 1, score : [2.]
episode : 2, score : [0.]
episode : 3, score : [0.]
episode : 4, score : [0.]
episode : 5, score : [0.]
